In [ ]:
!pip install --user kaggle

In [2]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"username","key":"apikey"}
import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 97% 25.0M/25.7M [00:02<00:00, 21.6MB/s]
100% 25.7M/25.7M [00:02<00:00, 12.2MB/s]


In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.5 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
from transformers import pipeline, GPT2Tokenizer, TFAutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split

# Load data from Kaggle
# You need to first download the Kaggle API key and put it in the 'path/to/kaggle.json' file
# You can download the IMDB movie review dataset from Kaggle using the following command:
# !kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
# Then, you can extract the 'IMDB Dataset.csv' file and put it in the 'path/to/imdb_reviews.csv' file
df = pd.read_csv('imdb-dataset-of-50k-movie-reviews.zip')

# Preprocess data
df = df.dropna()
df = df.reset_index(drop=True)
df['sentiment'] = np.where(df['sentiment'] == 'positive', 1, 0)

# Split data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['review'], df['sentiment'], test_size=0.2, random_state=42)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [24]:
# Create sentiment analysis pipeline using GPT
gpt_sentiment = pipeline('text-classification', model='gpt2', tokenizer='gpt2')

# Perform sentiment analysis on test set
gpt_preds = [gpt_sentiment(text, truncation=True)[0]['label'] for text in test_texts[:20]]

gpt_preds_value = [1 if lab == 'LABEL_1' else 0 for lab in gpt_preds]
# Calculate accuracy of GPT model
gpt_accuracy = (gpt_preds_value == test_labels[:20]).mean()
print(f"GPT accuracy: {gpt_accuracy:.3f}")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT accuracy: 0.600


In [ ]:
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 6.5 MB/s eta 0:00:00


In [36]:
import tensorflow as tf


# Load pre-trained BERT model and tokenizer
bert_model = TFAutoModelForSequenceClassification.from_pretrained('bert-base-uncased')
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

bert_sentiment = pipeline("sentiment-analysis", model=bert_model, tokenizer=bert_tokenizer)

bert_preds = [bert_sentiment(text)[0]['label'] for text in test_texts[:20]]
bert_preds_value = [1 if lab == 'LABEL_1' else 0 for lab in bert_preds]
# Calculate accuracy of BERT model
bert_accuracy = (bert_preds_value == test_labels[:20]).mean()
print(f"BERT accuracy: {bert_accuracy:.3f}")

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Token indices sequence length is longer than the specified maximum sequence length for this model (798 > 512). Running this sequence through the model will result in indexing errors


BERT accuracy: 0.550


In [35]:
bert_preds_value

[0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1]

In [44]:
# Create sentiment analysis pipeline using T5
t5_sentiment = pipeline('text2text-generation', model='t5-base', tokenizer='t5-base')

def t5_sentiment_analysis(text):
    output = t5_sentiment(f'Do text classification:\n {text} \n\n sentiment:')[0]['generated_text']
    return 1 if output == 'True' else 0

# Perform sentiment analysis on test set
t5_preds = [t5_sentiment_analysis(text) for text in test_texts[:20]]

# Calculate accuracy of T5 model
t5_accuracy = (t5_preds == test_labels[:20]).mean()
print(f"T5 accuracy: {t5_accuracy:.3f}")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


True
True
False


Token indices sequence length is longer than the specified maximum sequence length for this model (534 > 512). Running this sequence through the model will result in indexing errors


True
True
roland martin: "Tommy Boy" is a great movie and
True
True
False
br />br />br />
False
True
False
False
False
True
a heist artist wannabe and current 'hot' property Anna Maglo
True
True
False
T5 accuracy: 0.800
